# ReadMatch – One-click dev runner

This notebook installs dependencies and starts both the Python backend and the React frontend.

It is designed to replicate:

Terminal 1:
```bash
python3 app.py
```

Terminal 2:
```bash
cd frontend
npm start
```

## Notes
- This assumes you run the notebook from the project root (the folder that contains `app.py` and `frontend/`).
- The frontend is assumed to be a Create React App/Vite-style dev server that defaults to `http://localhost:3000`.
- If your backend uses a different port, adjust the URL in the **Open the app** cell.
- To stop both servers, run the **Stop servers** cell.


In [1]:
import os, sys, subprocess, textwrap, pathlib, time, threading, signal

PROJECT_ROOT = pathlib.Path('.').resolve()
FRONTEND_DIR = PROJECT_ROOT / 'frontend'

print('Project root:', PROJECT_ROOT)
print('Frontend dir exists:', FRONTEND_DIR.exists())
print('Python executable:', sys.executable)


Project root: /Users/elizabethsu/CS 410/ReadMatch
Frontend dir exists: True
Python executable: /opt/anaconda3/envs/cs410/bin/python


## 1) Install Python dependencies

This will prefer a `requirements.txt` if you have one. If not, it will attempt a minimal install for common Flask-style apps.


In [2]:
req = PROJECT_ROOT / 'requirements.txt'

if req.exists():
    print('Installing from requirements.txt...')
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-r', str(req)])
else:
    print('No requirements.txt found. Installing a minimal set of common deps...')
    minimal = [
        'flask',
        'flask-cors',
        'pandas',
        'numpy',
        'sqlalchemy',
        'scikit-learn',
        'tqdm',
        'requests'
    ]
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', *minimal])

print('Python deps installed.')


No requirements.txt found. Installing a minimal set of common deps...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 30.4 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [flask-cors]7 [flask]-learn]
Python deps installed.


## 2) Install Node dependencies

Runs `npm install` inside `frontend/` if `package.json` exists.


In [3]:
pkg = FRONTEND_DIR / 'package.json'
if not FRONTEND_DIR.exists():
    raise FileNotFoundError('frontend/ directory not found. Make sure you run this notebook from the project root.')
if not pkg.exists():
    raise FileNotFoundError('frontend/package.json not found.')

print('Installing npm dependencies...')
subprocess.check_call(['npm', 'install'], cwd=str(FRONTEND_DIR))
print('Node deps installed.')


Installing npm dependencies...


npm warn deprecated inflight@1.0.6: This module is not supported, and leaks memory. Do not use it. Check out lru-cache if you want a good and tested way to coalesce async requests by a key value, which is much more comprehensive and powerful.
npm warn deprecated glob@7.2.3: Glob versions prior to v9 are no longer supported
npm warn deprecated workbox-cacheable-response@6.6.0: workbox-background-sync@6.6.0
npm warn deprecated workbox-google-analytics@6.6.0: It is not compatible with newer versions of GA starting with v4, as long as you are using GAv3 it should be ok, but the package is not longer being maintained
npm warn deprecated w3c-hr-time@1.0.2: Use your platform's native performance.now() and performance.timeOrigin.
npm warn deprecated stable@0.1.8: Modern JS already guarantees Array#sort() is a stable sort, so this library is deprecated. See the compatibility table on MDN: https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Array/sort#browser_compati


added 1316 packages, and audited 1317 packages in 10s

267 packages are looking for funding
  run `npm fund` for details

9 vulnerabilities (3 moderate, 6 high)

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
Node deps installed.


## 3) Start backend + frontend (concurrently)

This cell starts both servers in the background using `subprocess.Popen`.
Logs will stream into the notebook output.


In [4]:
backend_proc = None
frontend_proc = None

def _stream(prefix, pipe):
    try:
        for line in iter(pipe.readline, ''):
            if not line:
                break
            print(f'[{prefix}] {line}', end='')
    finally:
        try:
            pipe.close()
        except Exception:
            pass

def start_servers():
    global backend_proc, frontend_proc
    
    if backend_proc and backend_proc.poll() is None:
        print('Backend already running.')
    else:
        print('Starting backend: python3 app.py')
        backend_proc = subprocess.Popen(
            [sys.executable, 'app.py'],
            cwd=str(PROJECT_ROOT),
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            bufsize=1,
        )
        threading.Thread(target=_stream, args=('backend', backend_proc.stdout), daemon=True).start()
    
    if frontend_proc and frontend_proc.poll() is None:
        print('Frontend already running.')
    else:
        print('Starting frontend: npm start')
        env = os.environ.copy()
        # Prevent CRA from trying to be interactive in some notebook terminals
        env.setdefault('CI', 'false')
        frontend_proc = subprocess.Popen(
            ['npm', 'start'],
            cwd=str(FRONTEND_DIR),
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            bufsize=1,
            env=env,
        )
        threading.Thread(target=_stream, args=('frontend', frontend_proc.stdout), daemon=True).start()

start_servers()
print('Servers launching...')
print('Next, run the "Open the app" cell.')


Starting backend: python3 app.py
Starting frontend: npm start
Servers launching...
Next, run the "Open the app" cell.


## 4) Open the app

If your React dev server runs on a different port (e.g., 5173 for Vite), change the URL.


In [5]:
import webbrowser, time

# Give the dev servers a brief moment to start.
time.sleep(3)

FRONTEND_URL = 'http://localhost:3000'
print('Opening:', FRONTEND_URL)
webbrowser.open(FRONTEND_URL)


[frontend] 
[frontend] > frontend@0.1.0 start
[frontend] > react-scripts start
[frontend] 
[backend] Traceback (most recent call last):
[backend]   File "/Users/elizabethsu/CS 410/ReadMatch/app.py", line 7, in <module>
[backend]     import retrieval
[backend]   File "/Users/elizabethsu/CS 410/ReadMatch/retrieval.py", line 1, in <module>
[backend]     from rank_bm25 import BM25Okapi
[backend] ModuleNotFoundError: No module named 'rank_bm25'
[frontend] (node:43015) [DEP_WEBPACK_DEV_SERVER_ON_AFTER_SETUP_MIDDLEWARE] DeprecationWarning: 'onAfterSetupMiddleware' option is deprecated. Please use the 'setupMiddlewares' option.
[frontend] (Use `node --trace-deprecation ...` to show where the warning was created)
[frontend] (node:43015) [DEP_WEBPACK_DEV_SERVER_ON_BEFORE_SETUP_MIDDLEWARE] DeprecationWarning: 'onBeforeSetupMiddleware' option is deprecated. Please use the 'setupMiddlewares' option.
[frontend] Starting the development server...
[frontend] 
Opening: http://localhost:3000


True

## 5) Stop servers

Run this cell when you're done.


In [6]:
def _terminate(proc, name):
    if not proc:
        print(f'{name}: not started.')
        return
    if proc.poll() is not None:
        print(f'{name}: already stopped.')
        return
    print(f'Stopping {name}...')
    try:
        # Try graceful terminate first
        proc.terminate()
        try:
            proc.wait(timeout=5)
        except Exception:
            proc.kill()
    except Exception as e:
        print(f'Error stopping {name}:', e)

_terminate(frontend_proc, 'frontend')
_terminate(backend_proc, 'backend')


Stopping frontend...
backend: already stopped.


[frontend] Compiled successfully!
[frontend] 
[frontend] You can now view frontend in the browser.
[frontend] 
[frontend]   Local:            http://localhost:3000
[frontend]   On Your Network:  http://192.168.10.226:3000
[frontend] 
[frontend] Note that the development build is not optimized.
[frontend] To create a production build, use npm run build.
[frontend] 
[frontend] webpack compiled successfully
